In [1]:
import tkinter as tk
from tkinter import ttk, filedialog
import pandas as pd
from pathlib import Path
import sys
from tkinter import font as tkfont

class DataFrameManager:
    def __init__(self, root):
        self.root = root
        self.root.title("DataFrame Column Manager")
        self.root.geometry("800x600")
        self.root.configure(bg='#1e1e1e')
        
        # Configure style
        self.style = ttk.Style()
        self.style.theme_use('default')
        self.style.configure('Custom.TButton',
                           background='#333333',
                           foreground='white',
                           padding=10,
                           font=('Roboto', 10))
        
        self.style.configure('Custom.TCheckbutton',
                           background='#1e1e1e',
                           foreground='white',
                           font=('Roboto', 10))
        
        # Load Roboto font
        try:
            self.roboto = tkfont.Font(family="Roboto", size=10)
        except:
            self.roboto = tkfont.Font(family="Arial", size=10)  # Fallback font
            
        # Initialize variables
        self.df = None
        self.selected_columns = []
        
        self.create_widgets()
        
    def create_widgets(self):
        # Main container
        main_container = tk.Frame(self.root, bg='#1e1e1e', padx=20, pady=20)
        main_container.pack(fill=tk.BOTH, expand=True)
        
        # File selection section
        file_frame = tk.Frame(main_container, bg='#1e1e1e')
        file_frame.pack(fill=tk.X, pady=(0, 20))
        
        self.file_label = tk.Label(
            file_frame,
            text="Select a CSV or Excel file:",
            bg='#1e1e1e',
            fg='white',
            font=self.roboto
        )
        self.file_label.pack(side=tk.LEFT)
        
        self.browse_button = tk.Button(
            file_frame,
            text="Browse",
            command=self.load_file,
            bg='#333333',
            fg='white',
            font=self.roboto,
            relief=tk.FLAT,
            padx=15,
            pady=5
        )
        self.browse_button.pack(side=tk.RIGHT)
        
        # Column selection section
        self.columns_frame = tk.Frame(main_container, bg='#1e1e1e')
        self.columns_frame.pack(fill=tk.BOTH, expand=True)
        
        self.columns_label = tk.Label(
            self.columns_frame,
            text="Available Columns:",
            bg='#1e1e1e',
            fg='white',
            font=self.roboto
        )
        self.columns_label.pack(anchor=tk.W)
        
        # Scrollable column list
        self.canvas = tk.Canvas(
            self.columns_frame,
            bg='#1e1e1e',
            highlightthickness=0
        )
        self.scrollbar = ttk.Scrollbar(
            self.columns_frame,
            orient="vertical",
            command=self.canvas.yview
        )
        self.scrollable_frame = tk.Frame(
            self.canvas,
            bg='#1e1e1e'
        )
        
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )
        
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        
        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")
        
        # Action buttons
        button_frame = tk.Frame(main_container, bg='#1e1e1e')
        button_frame.pack(fill=tk.X, pady=(20, 0))
        
        self.select_all_button = tk.Button(
            button_frame,
            text="Select All",
            command=self.select_all_columns,
            bg='#333333',
            fg='white',
            font=self.roboto,
            relief=tk.FLAT,
            padx=15,
            pady=5
        )
        self.select_all_button.pack(side=tk.LEFT)
        
        self.deselect_all_button = tk.Button(
            button_frame,
            text="Deselect All",
            command=self.deselect_all_columns,
            bg='#333333',
            fg='white',
            font=self.roboto,
            relief=tk.FLAT,
            padx=15,
            pady=5
        )
        self.deselect_all_button.pack(side=tk.LEFT, padx=10)
        
        self.save_button = tk.Button(
            button_frame,
            text="Save Selected Columns",
            command=self.save_selected_columns,
            bg='#4CAF50',  # Green color for primary action
            fg='white',
            font=self.roboto,
            relief=tk.FLAT,
            padx=15,
            pady=5
        )
        self.save_button.pack(side=tk.RIGHT)
        
        # Status bar
        self.status_label = tk.Label(
            main_container,
            text="",
            bg='#1e1e1e',
            fg='#4CAF50',
            font=self.roboto,
            anchor=tk.W
        )
        self.status_label.pack(fill=tk.X, pady=(20, 0))
        
    def load_file(self):
        file_path = filedialog.askopenfilename(
            filetypes=[
                ("CSV files", "*.csv"),
                ("Excel files", "*.xlsx *.xls")
            ]
        )
        
        if file_path:
            try:
                if file_path.endswith('.csv'):
                    self.df = pd.read_csv(file_path)
                else:
                    self.df = pd.read_excel(file_path)
                    
                self.update_column_list()
                self.status_label.config(
                    text=f"Loaded file with {len(self.df.columns)} columns"
                )
            except Exception as e:
                self.status_label.config(
                    text=f"Error loading file: {str(e)}",
                    fg='#ff4444'
                )
    
    def update_column_list(self):
        # Clear existing checkboxes
        for widget in self.scrollable_frame.winfo_children():
            widget.destroy()
        
        # Reset selected columns
        self.selected_columns = []
        self.column_vars = {}
        
        # Create new checkboxes
        for col in self.df.columns:
            var = tk.BooleanVar()
            self.column_vars[col] = var
            
            checkbox = tk.Checkbutton(
                self.scrollable_frame,
                text=col,
                variable=var,
                bg='#1e1e1e',
                fg='white',
                selectcolor='#333333',
                activebackground='#1e1e1e',
                activeforeground='white',
                font=self.roboto,
            )
            checkbox.pack(anchor=tk.W, pady=2)
    
    def select_all_columns(self):
        if self.df is not None:
            for var in self.column_vars.values():
                var.set(True)
    
    def deselect_all_columns(self):
        if self.df is not None:
            for var in self.column_vars.values():
                var.set(False)
    
    def save_selected_columns(self):
        if self.df is not None:
            selected_cols = [
                col for col, var in self.column_vars.items() if var.get()
            ]
            
            if not selected_cols:
                self.status_label.config(
                    text="Please select at least one column",
                    fg='#ff4444'
                )
                return
            
            file_path = filedialog.asksaveasfilename(
                defaultextension=".csv",
                filetypes=[
                    ("CSV files", "*.csv"),
                    ("Excel files", "*.xlsx")
                ]
            )
            
            if file_path:
                try:
                    new_df = self.df[selected_cols]
                    
                    if file_path.endswith('.csv'):
                        new_df.to_csv(file_path, index=False)
                    else:
                        new_df.to_excel(file_path, index=False)
                        
                    self.status_label.config(
                        text=f"Saved {len(selected_cols)} columns to {Path(file_path).name}",
                        fg='#4CAF50'
                    )
                except Exception as e:
                    self.status_label.config(
                        text=f"Error saving file: {str(e)}",
                        fg='#ff4444'
                    )

def main():
    root = tk.Tk()
    app = DataFrameManager(root)
    root.mainloop()

if __name__ == "__main__":
    main()